In [7]:
from dotenv import load_dotenv
from xcube_geodb.core.geodb import GeoDBClient

import os

from ingestion.flox_data_reader import DataReader

load_dotenv()

collection_name = os.environ["FTP_USER"] + "-raw"
f_collection_name = f"{os.environ['FTP_USER']}-raw-f"

server_url = (
    os.environ["GEODB_SERVER_URL"]
    if "GEODB_SERVER_URL" in os.environ
    else "https://xcube-geodb.brockmann-consult.de"
)
server_port = (
    os.environ["GEODB_SERVER_PORT"] if "GEODB_SERVER_PORT" in os.environ else 443
)
client_id = os.environ["GEODB_CLIENT_ID"]
client_secret = os.environ["GEODB_CLIENT_SECRET"]
auth_audience = (
    os.environ["GEODB_AUTH_AUD"]
    if "GEODB_AUTH_AUD" in os.environ
    else "https://xcube-users.brockmann-consult.de/api/v2"
)

geodb = GeoDBClient(
    server_url, server_port, client_id, client_secret, auth_aud=auth_audience
)
if not geodb.database_exists("deflox"):
    geodb.create_database("deflox")
if geodb.collection_exists(collection_name, "deflox"):
    geodb.drop_collection(collection_name, "deflox")
if geodb.collection_exists(f_collection_name, "deflox"):
    geodb.drop_collection(f_collection_name, "deflox")
properties = {
        collection_name: {
            "crs": 4326,
            "properties": {
                "local_datetime": "timestamp",
                "utc_datetime": "timestamp",
                "wr": "integer[]",
                "veg": "integer[]",
                "wr2": "integer[]",
                "DC_WR": "integer[]",
                "DC_VEG": "integer[]",
                "IT_WR[us]": "float",
                "IT_VEG[us]": "float",
                "cycle_duration[ms]": "float",
                "QEpro_Frame[C]": "float",
                "QEpro_CCD[C]": "float",
                "chamber_temp[C]": "float",
                "chamber_humidity": "float",
                "mainboard_temp[C]": "float",
                "mainboard_humidity": "float",
                "flox_identifier": "text",
                "GPS_TIME_UTC": "float",
                "GPS_date": "int",
                "GPS_lat": "float",
                "GPS_lon": "float",
                "voltage": "float",
                "gps_CPU": "float",
                "wr_CPU": "float",
                "veg_CPU": "float",
                "wr2_CPU": "float",
                "cooling_active": "text",
                "heating_active": "text",
                "Temp0": "float",
                "Temp1": "float",
                "Temp2": "float",
                "MultiCal": "float",
            },
        },
    f_collection_name: {
            "crs": 4326,
            "properties": {
                "local_datetime": "timestamp",
                "utc_datetime": "timestamp",
                "wr": "integer[]",
                "veg": "integer[]",
                "wr2": "integer[]",
                "DC_WR": "integer[]",
                "DC_VEG": "integer[]",
                "IT_WR[us]": "float",
                "IT_VEG[us]": "float",
                "cycle_duration[ms]": "float",
                "mainboard_temp[C]": "float",
                "mainboard_humidity": "float",
                "flox_identifier": "text",
                "GPS_TIME_UTC": "float",
                "GPS_date": "int",
                "GPS_lat": "float",
                "GPS_lon": "float",
                "voltage": "float",
                "gps_CPU": "float",
                "wr_CPU": "float",
                "veg_CPU": "float",
                "wr2_CPU": "float",
                "MultiCal": "float",
                "RSSI": "float",
            },
        }
}
geodb.create_collections(collections=properties, database="deflox")

In [ ]:
with open(
        "C:\\Users\\Thomas\\projects\\deflox\\test\\res\\240102\\070102.CSV", "r"
) as csvfile:
    gdf = DataReader().read(csvfile.readlines())
geodb.insert_into_collection(
    collection=collection_name, values=gdf, database="deflox"
)